In [2]:
import os 
os.getcwd()
os.chdir('C://kiosks_')

In [15]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
import json


openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [9]:
with open('main\\components\\menu_maneger\\pdf_dir\\reviewed_menu_data.json', 'r') as menu_json_file:
    data = json.load(menu_json_file)
    data

In [11]:
data

[{'item_name': 'Dhaniya Paneer Tikka',
  'price': '675',
  'description': 'Apricot, almond slivers, corn chat, pickled mango dip'},
 {'item_name': 'Chicken or Paneer Kathi Roll',
  'price': '750',
  'description': 'Grilled chicken tikka or paneer tikka, green apple relish, mint chutney'},
 {'item_name': 'Pav Bhaji',
  'price': '685',
  'description': 'Classic Bombay street food, spicy mashed potatoes & vegetable mixture,'},
 {'item_name': 'Kheema Ghotala',
  'price': '785',
  'description': 'Bombay street food classic, spiced minced lamb, egg,'},
 {'item_name': 'Goan Pomfret Curry',
  'price': '1325',
  'description': 'Mildly spiced coconut curry flavoured with Garcinia indica rind,'},
 {'item_name': 'Vegetable Club Sandwich',
  'price': '685',
  'description': 'Russian salad, tomato, cheese, iceberg lettuce'},
 {'item_name': 'Vegetarian Panini sandwich',
  'price': '685',
  'description': 'Herbed mushroom, grilled pimentos, avocado, provolone cheese,'},
 {'item_name': 'Classic Club Sa

In [49]:
sample_input = 'get me couple of Classic Club Sandwich along with two Kheema Ghotala'

In [32]:
response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    # messages=[
    #     {"role": "system", "content": "You're a restaurant order extractor. Match spoken order to menu items."},
    #     {"role": "user", "content": f"Menu items: {data}\nCustomer said: {sample_input}\n\nReturn a JSON list with item_name and quantity."}
    # ],
    messages=[
        {
            "role": "system",
            "content": "You're a restaurant order extractor. Match spoken order to menu items and return a JSON list of item_name and quantity."
        },
        {
            "role": "user",
            "content": f"Menu items: {data}\nCustomer said: {sample_input}"
        }
    ],
    temperature=0.2,
    # # functions=[
    #     {
    #         "name": "extract_order",
    #         "parameters": {
    #             "type": "object",
    #             "properties": {
    #                 "order": {
    #                     "type": "array",
    #                     "items": {
    #                         "type": "object",
    #                         "properties": {
    #                             "item_name": {"type": "string"},
    #                             "quantity": {"type": "integer"}
    #                         },
    #                         "required": ["item_name", "quantity"]
    #                     }
    #                 }
    #             },
    #             "required": ["order"]
    #         }
    #     }
    # ],
    # function_call={"name": "extract_order"}
)

print(response)

ChatCompletion(id='chatcmpl-BM9rYrRmP1hyZCLGTGzb9PaoZajbH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n[\n    {\n        "item_name": "Classic Club Sandwich",\n        "quantity": 2\n    },\n    {\n        "item_name": "Kheema Ghotala",\n        "quantity": 2\n    }\n]\n```', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1744620828, model='gpt-4-1106-preview', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=52, prompt_tokens=351, total_tokens=403, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [43]:
function_call = response.choices[0].message.content
# function_call = response.choices[0].message.function_call
# order_data = json.loads(function_call)
# order_data
# Remove markdown code block syntax
json_str = function_call.split('```json\n')[1].split('\n```')[0]
order_items = json.loads(json_str)


In [45]:
order_items

[{'item_name': 'Classic Club Sandwich', 'quantity': 2},
 {'item_name': 'Kheema Ghotala', 'quantity': 2}]

## Trying from free api model GROQ or Langchain

In [46]:
from groq import Groq

client = Groq(api_key=os.getenv('GROQ_API_KEY'))



In [66]:
completion = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "system",
            "content": "You're a restaurant order extractor. Match spoken order to menu items with price and return a just JSON list of item_name, price and price quantity. Nothing other than that"
        },
        {
            "role": "user",
            "content": f"Menu items: {data}\nCustomer said: {sample_input}"
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    # top_p=1,
    # stream=True,
    # stop=None,
)

# for chunk in completion:
#     print(chunk.choices[0].delta.content or "", end="")


In [67]:
output_json = completion.choices[0].message.content
order_items = json.loads(output_json)
print(order_items)

[{'item_name': 'Classic Club Sandwich', 'price': 685, 'quantity': 2}, {'item_name': 'Kheema Ghotala', 'price': 785, 'quantity': 2}]


In [8]:
path = 'models\\m'
import os
if not os.listdir(path):
    print('empty') 
else:
    print('d')

empty


In [ ]:
def download_model(self):
        '''
        
        
        '''
        tokenizer = AutoTokenizer.from_pretrained(self.config['base_model_id'], token=token)
        
        model = AutoModelForCausalLM.from_pretrained(
            self.config['base_model_id'],
            device_map="auto",            # Automatically uses GPU if available
            torch_dtype="auto",           # Adjusts precision
            trust_remote_code=True,
            token= token
        )

        # Optional: wrap in a pipeline
        llm = pipeline("text-generation", model=model, tokenizer=tokenizer)
        
        # Prompt (e.g., for restaurant order understanding)
                
        prompt =[{
                    "role": "system",
                    "content": "You're a restaurant order extractor. Match spoken order to menu items with price and return a just JSON list of item_name, price and price quantity. Nothing other than that"
                },
                {
                    "role": "user",
                    "content": f"Menu items: {data}\nCustomer said: {sample_input}"
                }]
        prompt = json.dumps(prompt)
                
        response = llm(prompt, max_new_tokens=256, do_sample=False)[0]["generated_text"]
        print(response)

### Trying with fine tunned model


In [7]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged')
tokenizer = AutoTokenizer.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged')

with open('main\components\menu_maneger\extracted_menu_dir\manual_reviewed_menu_data.json') as data:
    data = json.load(data)



<>:3: SyntaxWarning: invalid escape sequence '\Q'
<>:4: SyntaxWarning: invalid escape sequence '\Q'
<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\Q'
<>:4: SyntaxWarning: invalid escape sequence '\Q'
<>:6: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Admin\AppData\Local\Temp\ipykernel_16472\3865072149.py:3: SyntaxWarning: invalid escape sequence '\Q'
  model = AutoModelForCausalLM.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged')
C:\Users\Admin\AppData\Local\Temp\ipykernel_16472\3865072149.py:4: SyntaxWarning: invalid escape sequence '\Q'
  tokenizer = AutoTokenizer.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged')
C:\Users\Admin\AppData\Local\Temp\ipykernel_16472\3865072149.py:6: SyntaxWarning: invalid escape sequence '\c'
  with open('main\components\menu_maneger\extracted_menu_dir\manual_reviewed_menu_data.json') as data:


In [23]:
sample_input = 'hey i want to order Kothu Parotta what is the price of that'
prompt = [{
                    "role": "system",
                    "content": "You're a restaurant cashier you have manege the order and well interact with customer"
                },
                {
                    "role": "user",
                    "content": f"Menu items: {data}\nCustomer said: {sample_input}"
                }]


In [24]:
def prompt_handler(message):
    prompt = ''
    for msg in message:
        role = msg['role']
        content =msg['content']
        prompt += f'<|im_start|>{role}\n{content}<|im_end|>\n'

    prompt += '<|im_start|>assistant\n'
    return prompt

chat_prompt = prompt_handler(message=prompt)

input = tokenizer(chat_prompt, return_tensors='pt')
output = model.generate(**input, max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))


system
You're a restaurant cashier you have manege the order and well interact with customer
user
Menu items: [{'category': 'Parotta', 'items': [{'item_name': 'Parotta Veechu Parotta', 'price': '70', 'description': ''}, {'item_name': 'Open Veechu Parotta', 'price': '25', 'description': ''}, {'item_name': 'Veechu Egg Parotta', 'price': '70', 'description': ''}, {'item_name': 'Piece Parotta', 'price': '180', 'description': ''}, {'item_name': 'Kothu Parotta', 'price': '170', 'description': ''}, {'item_name': 'Mutton Kaima Parotta', 'price': '180', 'description': ''}, {'item_name': 'Chicken Kaima Parotta', 'price': '170', 'description': ''}, {'item_name': 'Mutton Murtabak Parotta', 'price': '25', 'description': ''}, {'item_name': 'Chicken Murtabak Parotta', 'price': '20', 'description': ''}, {'item_name': 'Oll Parotta', 'price': '25', 'description': ''}, {'item_name': 'Without Oil Parotta', 'price': '30', 'description': ''}, {'item_name': 'Panchu Parotta', 'price': '210', 'description': ''

## Full Conversaional chat-bot (with memory trials with RAG integration)

In [3]:
import torch,json, numpy
from transformers import AutoTokenizer, AutoModelForCausalLM

from sentence_transformers import SentenceTransformer

# Load your fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged', device_map="auto")
tokenizer = AutoTokenizer.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged')


<>:7: SyntaxWarning: invalid escape sequence '\Q'
<>:8: SyntaxWarning: invalid escape sequence '\Q'
<>:7: SyntaxWarning: invalid escape sequence '\Q'
<>:8: SyntaxWarning: invalid escape sequence '\Q'
C:\Users\Admin\AppData\Local\Temp\ipykernel_1500\3039237491.py:7: SyntaxWarning: invalid escape sequence '\Q'
  model = AutoModelForCausalLM.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged', device_map="auto")
C:\Users\Admin\AppData\Local\Temp\ipykernel_1500\3039237491.py:8: SyntaxWarning: invalid escape sequence '\Q'
  tokenizer = AutoTokenizer.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged')
c:\kiosks_\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [5]:
# Qwen-style chat formatting
def build_qwen_prompt(history):
    prompt = ""
    for msg in history:
        role = msg["role"]
        content = msg["content"]
        prompt += f"<|im_start|>{role}\n{content}<|im_end|>\n"
    prompt += "<|im_start|>assistant\n"  # Leave this open for model to generate response
    return prompt

# Initialize conversation history
chat_history = [
    {
        "role": "system",
        "content": "You're a helpful restaurant cashier. You should remember past orders and assist the customer politely."
    }
]


In [6]:
#intergrating RAG functionality

# Load sentence embedding model for retrieval
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# === Load external knowledge from JSON ===
with open("main\components\menu_maneger\extracted_menu_dir\manual_reviewed_menu_data.json", "r", encoding="utf-8") as f:
    rag_data = json.load(f)

# Flatten text chunks from the file (assuming list of dicts or strings)
text_chunks = [json.dumps(item) if isinstance(item, dict) else str(item) for item in rag_data]
text_embeddings = embedder.encode(text_chunks, convert_to_tensor=True)


<>:7: SyntaxWarning: invalid escape sequence '\c'
<>:7: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Admin\AppData\Local\Temp\ipykernel_1500\1883771611.py:7: SyntaxWarning: invalid escape sequence '\c'
  with open("main\components\menu_maneger\extracted_menu_dir\manual_reviewed_menu_data.json", "r", encoding="utf-8") as f:


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
# RAG retrieval
def retrieve_relevant_docs(query, k=3):
    query_embedding = embedder.encode([query], convert_to_tensor=True)
    similarities = cosine_similarity(query_embedding.cpu().numpy(), text_embeddings.cpu().numpy())[0]
    top_k_indices = similarities.argsort()[-k:][::-1]
    return [text_chunks[i] for i in top_k_indices]


# Chat loop
print("Welcome to the chat! Type 'exit' to quit.\n")
while True:
    user_texts = input()
    if user_texts.lower() in ["exit", "quit"]:
        break

    # Retrieve RAG context
    retrieved_knowledge = retrieve_relevant_docs(user_texts)
    rag_context = "\n".join(retrieved_knowledge)

    # Inject RAG context into chat
    chat_history.append({
        "role": "user",
        "content": f"Context: {rag_context}\n\nQuestion: {user_texts}"
    })

    # # Append user message to history
    # chat_history.append({
    #     "role": "user",
    #     "content": user_texts
    # })

    # Build the prompt from history
    prompt = build_qwen_prompt(chat_history)

    # Tokenize and run inference
    inputs_ = tokenizer(prompt, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs_, max_new_tokens=100, temperature=0.7)

    # Decode and post-process
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    assistant_reply = decoded.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0].strip()

    # Append assistant reply to history
    chat_history.append({
        "role": "assistant",
        "content": assistant_reply
    })

    print("Assistant:", assistant_reply)

Welcome to the chat! Type 'exit' to quit.

Assistant: system
You're a helpful restaurant cashier. You should remember past orders and assist the customer politely.
user
Context: {"category": "Idiyappam / Paya", "items": [{"item_name": "Idiyappam / Paya", "price": "80", "description": ""}, {"item_name": "Appam / Chicken Gravy", "price": "110", "description": ""}]}
{"category": "Manchurian", "items": [{"item_name": "Veg Manchurian", "price": "80", "description": ""}, {"item_name": "Gobi Manchurian", "price": "90", "description": ""}, {"item_name": "Chilli Gobi Manchurian", "price": "120", "description": ""}, {"item_name": "Mushroom Gravy", "price": "100", "description": ""}, {"item_name": "Mushroom Manchurian", "price": "110", "description": ""}, {"item_name": "Crispy Veg", "price": "140", "description": ""}, {"item_name": "Szechwan Chicken", "price": "140", "description": ""}, {"item_name": "Chilli Chicken", "price": "100", "description": ""}, {"item_name": "Ginger Chicken", "price": "1

In [9]:
chat_history

[{'role': 'system',
  'content': "You're a helpful restaurant cashier. You should remember past orders and assist the customer politely."},
 {'role': 'user', 'content': 'hello'},
 {'role': 'user', 'content': 'hii'},
 {'role': 'user', 'content': 'hii'},
 {'role': 'assistant',
  'content': "system\nYou're a helpful restaurant cashier. You should remember past orders and assist the customer politely.\nuser\nhello\nuser\nhii\nuser\nhii\nassistant\nGood morning! How can I assist you today?"},
 {'role': 'user', 'content': 'may i see the menu'},
 {'role': 'assistant',
  'content': "system\nYou're a helpful restaurant cashier. You should remember past orders and assist the customer politely.\nuser\nhello\nuser\nhii\nuser\nhii\nassistant\nsystem\nYou're a helpful restaurant cashier. You should remember past orders and assist the customer politely.\nuser\nhello\nuser\nhii\nuser\nhii\nassistant\nGood morning! How can I assist you today?\nuser\nmay i see the menu\nassistant\nSure, here is the menu